In [79]:
import numpy as np
from scipy.optimize import minimize


def score(r, d, params):
    w_r, w_d, gamma, r_missing, bias = params
    r_eff = np.where(r < 0, r_missing, r)
    rating_term = np.log1p(r_eff)
    dist_term = d ** gamma
    return w_r * rating_term - w_d * dist_term + bias


def m_loss(params):
    s_w = score(r_w, d_w, params)
    s_l = score(r_l, d_l, params)
    diff = s_l - s_w
    return np.mean(np.logaddexp(0.0, diff))


train_path = "data/327_restaurants_2/restaurants_train.txt"

r_w, d_w = [], []
r_l, d_l = [], []

with open(train_path) as f:
    for line in f:
        if not line.strip():
            continue
        winner, r1, r2, d1, d2 = map(float, line.split())

        if winner == 0.5:
            continue
        if winner == 0:
            r_w.append(r1); d_w.append(d1)
            r_l.append(r2); d_l.append(d2)
        else:
            r_w.append(r2); d_w.append(d2)
            r_l.append(r1); d_l.append(d1)

r_w = np.array(r_w)
d_w = np.array(d_w)
r_l = np.array(r_l)
d_l = np.array(d_l)


init = np.array([100.0, 10.0, 0.1, 5.0, 0.0])

bounds = [
    (0.0, None),
    (0.0, None),
    (1e-3, 1.0),
    (0.0, 10.0),
    (None, None),
]

res = minimize(
    m_loss,
    init,
    method="L-BFGS-B",
    bounds=bounds,
    options={"maxiter": 500}
)

print("Success:", res.success)
print("m =", res.fun)
print("Params:", [round(x, 3) for x in res.x])

Success: True
m = 0.5427131908499262
Params: [4.8, 44.4, 0.033, 5.704, 0.0]


In [59]:
import sys
import math


def main():
    w_r = 4.8
    w_d = 44.4
    gamma = 0.033
    impute = 5.7

    n = int(input())
    for _ in range(n):
        r, d = map(float, input().split())
        if r < 0:
            r = impute
        print(w_r * math.log1p(r) - w_d * (d ** gamma))


if __name__ == '__main__':
    main()